# Dataset Generator

In [ ]:
# default_exp dataset

In [ ]:
# export
from syntheticDataGen.randomParameters import NormalMixPNParameters2 as NMixPar

def GaussianMixtureDataGenerator(dim, n_comps, aucpn_range,irreducibility_params,timeoutMins=1,nTimeouts=5):
    #setting the thrid entry of irreducibility_params to False enforces pairwise mutual irreducibility
    #setting the thrid entry of irreducibility_params to True enforces the strong irreducibility.
    #The second entry of irreducibility_params is the the posterior threshold for irreducibility.
    #The first argument is the proportion of points that should high enough posterior.
    #Using 0.01 and 0.9 is easier to satisfy then 0.05 and 0.95. You may play with this,
    # but note that the stronger irreducibility criteria, the more difficult it is to be satisfied
    # and you might end up in infinite loop since no parameters could be founs to satisfy bo the
    # itrreducibility and auc criteria.
    NMix = NMixPar(dim, n_comps,aucpn_range, irreducibility_params,timeoutMins=timeoutMins,nTimeouts=nTimeouts)
#     NMix.perturb2SatisfyMetrics()
#     NMix.updateDataGenerator()
    dg = NMix.dg
    return NMix, dg

In [ ]:
# export
from syntheticDataGen.mixture import Mixture

def createMixture(NMix,cluster_num):
    gamma = NMix.alpha * NMix.p_pos[cluster_num] + (1 - NMix.alpha) * NMix.p_neg[cluster_num]
    eta = NMix.alpha * NMix.p_pos[cluster_num] / gamma
    m = Mixture((NMix.dg.dist_p.comps[cluster_num],
                 NMix.dg.dist_n.comps[cluster_num]),np.array([eta,1-eta]))
    return m

In [ ]:
# export
def pointsBetween(NMix,i,j):
    mixI = createMixture(NMix,i)
    mI = np.dot(mixI.mixProp,[c.mean for c in mixI.comps])
    vec = NMix.vecBetweenPair(i,j)
    for scale in np.arange(0,1,.01):
        yield mI - vec * scale

In [ ]:
# export
import numpy as np
def mycdf(mixture,x,n_samples=10000):
    mixtureMean = mixture.mean
    d = np.linalg.norm(mixtureMean - x)**2
    dists = np.linalg.norm(mixture.rvs(n_samples) - mixtureMean,axis=-1)**2
    return (dists <= d).sum() / n_samples

In [ ]:
# export
def clustersConflict(NMix,i,j):
    mixI = createMixture(NMix,i)
    mixJ = createMixture(NMix,j)
    for x in pointsBetween(NMix,i,j):
        if (1 - mycdf(mixI,x)) > .05 and (1-mycdf(mixJ,x)) > .05:
            return True
    return False

In [ ]:
# export
from itertools import combinations
import scipy.stats as ss

def jitterMixtures(NMix,i,j):
    jitterFactor = ss.gamma.rvs(2)
    print(i,j,jitterFactor)
    NMix.dg.components_pos[i].mean += jitterFactor
    NMix.dg.components_neg[i].mean += jitterFactor
    NMix.dg.components_pos[j].mean -= jitterFactor
    NMix.dg.components_neg[j].mean -= jitterFactor
    return NMix

def getConflictingPairs(NMix):
    return [(i,j) for i,j in combinations(np.arange(NMix.nComps),2) if clustersConflict(NMix,i,j)]

def jitterComponents(NMix):
    pairs = getConflictingPairs(NMix)
    while len(pairs):
        for i,j in pairs:
            while clustersConflict(NMix,i,j):
                NMix = jitterMixtures(NMix,i,j)
        pairs = getConflictingPairs(NMix)
    return NMix

def generateParams(n_clusters,dim,aucRange,irreducibility_params=[.01,0.9,False],timeoutMins=1,nTimeouts=5):
    NMix,dg = GaussianMixtureDataGenerator(dim,n_clusters,aucRange,irreducibility_params,timeoutMins=timeoutMins,nTimeouts=nTimeouts)
    NMix = jitterComponents(NMix)
    return NMix

In [ ]:
paramset = generateParams(3,16,[.75,.95])

  0%|                                                                                                                                                     | 0/3 [00:00<?, ?it/s]

isMetricSatisfied{'aucpn': 0.5, 'irreducibility': [0.0, 0.0]}
anchorSetProp0.01
Mu Perturb
isMetricSatisfied{'aucpn': 0.75269, 'irreducibility': [0.0, 0.0]}
anchorSetProp0.01
Sigma Scale Perturb
Sigma Ratios: [1.00000000e+00 1.52587891e-05]
0.75
Sigma Ratios: [1.        0.0100226]
0.875
Sigma Ratios: [1.         0.11806709]
0.9375
Sigma Ratios: [1.         0.35607413]
0.96875


 33%|███████████████████████████████████████████████                                                                                              | 1/3 [00:00<00:00,  2.67it/s]

Sigma Ratios: [1.        0.6017103]
Sigma Ratios: [1.        0.6017103]
isMetricSatisfied{'aucpn': 0.76002756, 'irreducibility': [0.0, 0.0]}
anchorSetProp0.01
Mu Perturb
isMetricSatisfied{'aucpn': 0.8699896, 'irreducibility': [0.0106, 0.0194]}
anchorSetProp0.01
isMetricSatisfied{'aucpn': 0.5, 'irreducibility': [0.0, 0.0]}
anchorSetProp0.01
Mu Perturb
isMetricSatisfied{'aucpn': 0.7656784799999999, 'irreducibility': [0.0, 0.0]}
anchorSetProp0.01
Mu Perturb
isMetricSatisfied{'aucpn': 0.7782407200000001, 'irreducibility': [0.0, 0.0004]}
anchorSetProp0.01
Mu Perturb
isMetricSatisfied{'aucpn': 0.85147036, 'irreducibility': [0.008, 0.01]}
anchorSetProp0.01
Perturb Proportion
isMetricSatisfied{'aucpn': 0.8590766400000002, 'irreducibility': [0.0086, 0.0108]}
anchorSetProp0.01
Perturb Proportion


 67%|██████████████████████████████████████████████████████████████████████████████████████████████                                               | 2/3 [00:01<00:00,  1.67it/s]

isMetricSatisfied{'aucpn': 0.8515144800000001, 'irreducibility': [0.009, 0.0098]}
anchorSetProp0.01
Mu Perturb
isMetricSatisfied{'aucpn': 0.9189236799999999, 'irreducibility': [0.083, 0.0838]}
anchorSetProp0.01
isMetricSatisfied{'aucpn': 0.5, 'irreducibility': [0.0, 0.0]}
anchorSetProp0.01
Sigma Scale Perturb
Sigma Ratios: [1.00000000e+00 1.52587891e-05]
0.75
Sigma Ratios: [1.        0.0100226]
0.875
Sigma Ratios: [1.         0.11806709]
0.9375
Sigma Ratios: [1.         0.35607413]
0.96875
Sigma Ratios: [1.        0.6017103]
Sigma Ratios: [1.        0.6017103]
isMetricSatisfied{'aucpn': 0.5317255599999999, 'irreducibility': [0.0, 0.0]}
anchorSetProp0.01
Sigma Scale Perturb
Sigma Ratios: [1.0000000e+00 9.1813706e-06]
0.75
Sigma Ratios: [1.        0.0060307]
0.875
Sigma Ratios: [1.         0.07104218]
0.9375
Sigma Ratios: [1.         0.21425347]
0.96875
Sigma Ratios: [1.         0.36205529]
0.984375
Sigma Ratios: [1.         0.46768846]
0.9921875
Sigma Ratios: [1.        0.5307469]
Sigma

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:02<00:00,  1.22it/s]

isMetricSatisfied{'aucpn': 0.84632204, 'irreducibility': [0.0044, 0.1272]}
anchorSetProp0.01
Mu Perturb
isMetricSatisfied{'aucpn': 0.89701664, 'irreducibility': [0.0342, 0.1518]}
anchorSetProp0.01


0 1 3.78094857357646


In [ ]:
paramset.dg.pn_data(1000)[0].shape

(1000, 16)